# Battle of Neighborhoods
## 1. Introduction / Business problem 

Each year number of practicing dentists in Florida grows by average by 170 [(source)](https://www.ada.org/en/science-research/health-policy-institute/data-center/supply-and-profile-of-dentists).
![](fl_dentists_chart.png)

Most of new dentists start as Associates in the established practices, but finally they want to open their own offices. 

Here comes the problem - what is the best place to open the new dental practice?

We will examine in this exercise a naïve approach of comparing population numbers in a given neighborhood to density and quality of existing dental practices. Places with least dental practice density per population will be considered as candidate spots.

This approach might be further expanded by comparing additional data as wealth (e.g. median household income) and commercial rent rates.

I am narrowing the scope of the exercise to Miami metropolitan area, with focus on Miami-Dade county.

## 2. Data

The exercise will base on the following data sets:

1. Foursquare (the business type `Dentist's Office`) - location and popularity
2. Geographical / geocoding data [zip code boundaries](https://gis-mdc.opendata.arcgis.com/datasets/zip-code), [KML file with boundaries](https://opendata.arcgis.com/datasets/fee863cb3da0417fa8b5aaf6b671f8a7_0.kml?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D)

3. Demographics [by zip code](https://worldpopulationreview.com/zips/florida/)

4. Zip Code Characteristics: Mean and Median Household Income [xlsx](https://www.psc.isr.umich.edu/dis/census/Features/tract2zip/MedianZIP-3.xlsx)


Results will name best place(s) to open Dental Office in Miami area. 

### 2.1. Data wrangling

Geographical data in form of `KLM` file was downloaded from `arcgis.com` site. The file was converted to `.topojson` format using [MyGeoData on-line converter](https://mygeodata.cloud/converter/). `Zip_Code.topojson` file is natively recognized by `folium` library to create chloropleth maps.
The `KML` file was also converted to `Zip_Code.tsv` using `bash` tools. The file has a following structure:
```
ZIP_CODE1<tab>point1.1<tab>point1.2<tab>...<tab>point1.N
ZIP_CODE2<tab>point2.1<tab>point2.2<tab>...<tab>point2.M
...
```
where `point` is a pair of latitude,longitude coordinates. The snippet below load the `Zip_Code.tsv` file into a dictionary `zip_borders`.

In [ ]:
#creates a dict where key is ZIP and value is list of border points 
#point is a tuple (lat, lng)
zip_borders = {}
with open('Zip_Code.tsv', 'r') as f:
    for line in f:
        line = line.strip().split('\t')
        coords = []
        for coord in line[1:]:
            coords.append(
                (float(coord.split(',')[1]), #lat and lng are swapped!!!
                 float(coord.split(',')[0]))
            )
        zip_borders[line[0]] = coords

Population by ZIP code was downloaded in `.csv` format from [worldpopulationreview.com](https://worldpopulationreview.com/zips/florida/) as file `population_by_zip.csv`. This data was directly imported into Pandas DataFrame, then converted to integers.

In [ ]:
import pandas as pd
pop_df = pd.read_csv('population_by_zip.csv')
pop_df = pop_df.astype({'zip': 'int32', 'pop': 'int32'})
pop_df = pop_df.set_index('zip')

Household Median Income was downloaded from [umich.edu](https://www.psc.isr.umich.edu/dis/census/Features/tract2zip/MedianZIP-3.xlsx). The `XLSX` file was manually converted to `MedianZIP-3.csv` and loaded do `zip_income` DataFrame.

In [ ]:
zip_income = pd.read_csv('MedianZIP-3.csv').set_index('Zip')

### 2.2 Reading venue data from Foursquare

The right endpoint and API call for this excercise was [venues/search](https://developer.foursquare.com/docs/api-reference/venues/search/). It requires latitude, longitude and radius for the queried area with additional filtering possible. Here we used [filtering by category](https://developer.foursquare.com/docs/build-with-foursquare/categories/). As we wanted to get a list of all Dental Offices in the Greater Miami area, we might call the API just once with a large enough radius to cover it all. Unfortunately the hard limit for returned results in Foursquare API is 50, so we turned to querying each ZIP code individually.

The function `get_dentists(lat, lng, r)` returns DataFrame with all returned offices in a given radius.

In [ ]:
def get_dentists(lat, lng, r):
    #get Dentist's Offices from foursquare
    CLIENT_ID = 'W4Z5L01JHWF0ARMJNDDVE5KOBVNQDVQON1RN15MXW1HOTVYP' # your Foursquare ID
    CLIENT_SECRET = 'MKRMHSSBSBESO33A2EJDA0ZORVBGC4AHVYO4IAO1JU0XGVXG' # your Foursquare Secret
    VERSION = '20200601' # Foursquare API version
    LIMIT = 50 # limit of number of venues returned by Foursquare API
    #"Dentist's Office" categoryID 
    #(from https://developer.foursquare.com/docs/build-with-foursquare/categories/)
    dentist_cat = '4bf58dd8d48988d178941735'

    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        r, 
        LIMIT,
        dentist_cat
        )
    results = requests.get(url).json()
    
    #create dental office array
    offices = []
    for v in results['response']['venues']:
        name = v['name']
        lat = v['location']['lat']
        lng = v['location']['lng']
        try:
            ZIP = v['location']['postalCode']
        except KeyError:
            ZIP = None
        try:
            address = ', '.join(v['location']['formattedAddress'])
        except KeyError:
            address = None
        offices.append({
            'name': name,
            'lat': lat,
            'lng': lng,
            'zip': ZIP,
            'address': address
        })
    return pd.DataFrame(offices)

Now we need to run the above function over all ZIP areas.
The dictionary `zip_borders` contains coordinates of points making a border of a ZIP area. So, we need to find a smallest enclosing circle for each set of points. 

The small open source library below fits this purpose. 

In [ ]:
# 
# Smallest enclosing circle - Library (Python)
# 
# Copyright (c) 2020 Project Nayuki
# https://www.nayuki.io/page/smallest-enclosing-circle
# 
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# 
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
# 
# You should have received a copy of the GNU Lesser General Public License
# along with this program (see COPYING.txt and COPYING.LESSER.txt).
# If not, see <http://www.gnu.org/licenses/>.
# 

import math, random


# Data conventions: A point is a pair of floats (x, y). A circle is a triple of floats (center x, center y, radius).

# Returns the smallest circle that encloses all the given points. Runs in expected O(n) time, randomized.
# Input: A sequence of pairs of floats or ints, e.g. [(0,5), (3.1,-2.7)].
# Output: A triple of floats representing a circle.
# Note: If 0 points are given, None is returned. If 1 point is given, a circle of radius 0 is returned.
# 
# Initially: No boundary points known
def make_circle(points):
	# Convert to float and randomize order
	shuffled = [(float(x), float(y)) for (x, y) in points]
	random.shuffle(shuffled)
	
	# Progressively add points to circle or recompute circle
	c = None
	for (i, p) in enumerate(shuffled):
		if c is None or not is_in_circle(c, p):
			c = _make_circle_one_point(shuffled[ : i + 1], p)
	return c


# One boundary point known
def _make_circle_one_point(points, p):
	c = (p[0], p[1], 0.0)
	for (i, q) in enumerate(points):
		if not is_in_circle(c, q):
			if c[2] == 0.0:
				c = make_diameter(p, q)
			else:
				c = _make_circle_two_points(points[ : i + 1], p, q)
	return c


# Two boundary points known
def _make_circle_two_points(points, p, q):
	circ = make_diameter(p, q)
	left  = None
	right = None
	px, py = p
	qx, qy = q
	
	# For each point not in the two-point circle
	for r in points:
		if is_in_circle(circ, r):
			continue
		
		# Form a circumcircle and classify it on left or right side
		cross = _cross_product(px, py, qx, qy, r[0], r[1])
		c = make_circumcircle(p, q, r)
		if c is None:
			continue
		elif cross > 0.0 and (left is None or _cross_product(px, py, qx, qy, c[0], c[1]) > _cross_product(px, py, qx, qy, left[0], left[1])):
			left = c
		elif cross < 0.0 and (right is None or _cross_product(px, py, qx, qy, c[0], c[1]) < _cross_product(px, py, qx, qy, right[0], right[1])):
			right = c
	
	# Select which circle to return
	if left is None and right is None:
		return circ
	elif left is None:
		return right
	elif right is None:
		return left
	else:
		return left if (left[2] <= right[2]) else right


def make_diameter(a, b):
	cx = (a[0] + b[0]) / 2
	cy = (a[1] + b[1]) / 2
	r0 = math.hypot(cx - a[0], cy - a[1])
	r1 = math.hypot(cx - b[0], cy - b[1])
	return (cx, cy, max(r0, r1))


def make_circumcircle(a, b, c):
	# Mathematical algorithm from Wikipedia: Circumscribed circle
	ox = (min(a[0], b[0], c[0]) + max(a[0], b[0], c[0])) / 2
	oy = (min(a[1], b[1], c[1]) + max(a[1], b[1], c[1])) / 2
	ax = a[0] - ox;  ay = a[1] - oy
	bx = b[0] - ox;  by = b[1] - oy
	cx = c[0] - ox;  cy = c[1] - oy
	d = (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by)) * 2.0
	if d == 0.0:
		return None
	x = ox + ((ax*ax + ay*ay) * (by - cy) + (bx*bx + by*by) * (cy - ay) + (cx*cx + cy*cy) * (ay - by)) / d
	y = oy + ((ax*ax + ay*ay) * (cx - bx) + (bx*bx + by*by) * (ax - cx) + (cx*cx + cy*cy) * (bx - ax)) / d
	ra = math.hypot(x - a[0], y - a[1])
	rb = math.hypot(x - b[0], y - b[1])
	rc = math.hypot(x - c[0], y - c[1])
	return (x, y, max(ra, rb, rc))


_MULTIPLICATIVE_EPSILON = 1 + 1e-14

def is_in_circle(c, p):
	return c is not None and math.hypot(p[0] - c[0], p[1] - c[1]) <= c[2] * _MULTIPLICATIVE_EPSILON


# Returns twice the signed area of the triangle defined by (x0, y0), (x1, y1), (x2, y2).
def _cross_product(x0, y0, x1, y1, x2, y2):
	return (x1 - x0) * (y2 - y0) - (y1 - y0) * (x2 - x0)

# Convert "(lat,lng)" radius to meters
# Length in meters of 1° of latitude = always 111.32 km
# below is by no means an accurate math, but it is good enough
# because we are working on a small scale where earth curvature can be ignored
def make_circle_metric(points):
    c = make_circle(points)
    r = c[2] * 111320
    return (c[0], c[1], r)

The original library was enhanced with `make_circle_metric` function that returns approximate radius in meters, as required by Foursquare API.

With the data desctibed above we can now create a map of Greater Miami Area that includes smallest enclosing circles of each ZIP area. 


In [ ]:
import folium
import branca
import json

zip_map = r'Zip_Code.topojson' # topojson file

#yellow --> green color scale from 0 to maximum population
colorscale = branca.colormap.linear.YlGn_09.scale(0, pop_df.max()['pop'])
with open(zip_map, mode='r') as f:
    topo = json.load(f)
    
#find zip codes present in the topojson file
topo_zips = set()
for g in topo['objects']['Zip_Code']['geometries']:
    topo_zips.add(g['properties']['ZIP'])

#zip codes in population file
pop_zips=set(pop_df['zip'])

#add missing zip codes to population file (with value = 0)
#this is necessary for folium to parse all zip areas
for z in topo_zips - pop_zips:
    pop_df = pop_df.append({'zip':z, 'pop':0}, ignore_index=True)

pop_series = pop_df.set_index('zip')['pop']
def style_function(feature):
    pop = pop_series.get(int(feature['properties']['ZIP']), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if pop is None else colorscale(pop)
    }

miami_area = [25.70, -80.208889]
m = folium.Map(
    location=miami_area,
    tiles='cartodbpositron',
    zoom_start=9
)
folium.TopoJson(
    topo,
    'objects.Zip_Code',
    style_function=style_function
).add_to(m)

Calculate and display smallest enclosing circles:

In [ ]:
for z in zip_borders:
    lat, lng, r = make_circle_metric(zip_borders[z])
    folium.Circle([lat, lng],
                    radius=r
                   ).add_to(m)
m

Now, call the `get_dentists` function for each ZIP area.

In [ ]:
#get dentists by zip codes
miami_dentists = {}
for i,z in enumerate(zip_borders):
    lat, lng, r = make_circle_metric(zip_borders[z])
    dent = get_dentists(lat, lng, r)
    try:
        dent = dent[dent['zip']==z] #remove offices from other ZIPs
    except KeyError:
        continue
    if len(dent) > 0: miami_dentists[z] = dent #remove zips with no dentists

This part of the exercise is prone to networking errors and takes some time, so the successful result is saved in a `pickle` format to avoid repeated calls.

In [ ]:
import pickle

with open('miami_dentists.pickle', 'wb') as f:
    pickle.dump(miami_dentists, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('miami_dentists.pickle', 'rb') as f:
    miami_dentists = pickle.load(f)

## 3. Data visualization

The following function adds Dental Offices to a map.

In [ ]:
def add_dentists_to_map(df,m, add_label=''):
    for lat, lng, label in zip(df['lat'], df['lng'], df['name']):
        label = folium.Popup(label+add_label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=2,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(m)

Now we can create a map with all Dental Offices in the Miami Area.

In [ ]:
m = folium.Map(
    location=miami_area,
    tiles='cartodbpositron',
    zoom_start=9
)
folium.TopoJson(
    topo,
    'objects.Zip_Code',
    style_function=style_function
).add_to(m)

for d in miami_dentists:
    add_dentists_to_map(miami_dentists[d],m, add_label=', ZIP:'+d)
m

## 4. Model creation

The following hypothesis was considered: number of Dental Offices in a given area is in relation to this area population and median household income.
The hypothesis was checked using multi linear regression.

Preparing an array with all collected data.

In [ ]:
# getting population, median income and number of dental offices per ZIP
# list of ZIPs is stored in 'zip_borders' dict
zip_data = []
for z in zip_borders:
    pop = pop_df.loc[int(z), 'pop']
    if pop > 0:
        if str(z) in miami_dentists:
            dentists_in_zip = len(miami_dentists[str(z)])
        else:
            dentists_in_zip = 0
        #number of practices per 10000 population
        ratio = dentists_in_zip/pop * 10000 
        income = int(zip_income['Median'][int(z)].replace(',' , '')) 
        zip_data.append({'zip': int(z), 
                         'pop': pop, 
                         'dentists': dentists_in_zip,
                         'income': income,
                         'ratio': ratio}) 
zip_data = pd.DataFrame(zip_data).set_index('zip')

In [ ]:
zip_data.head()

In [ ]:
'''
# display the whole dataframe
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(zip_data.sort_values(by=['ratio','pop'], ascending=False))
'''

In [ ]:
# use the inline backend to generate the plots within the browser
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt
# generate histogram
zip_data[['ratio']].plot(kind='hist', figsize=(10, 6))

plt.title('Histogram of Dental Office numbers in ZIP areas')
plt.ylabel('Number of ZIP areas')
plt.xlabel('Number of Dental Offices')

plt.show()

In [ ]:
zip_data.plot(kind='scatter', x='pop', y='dentists', figsize=(10, 6), color='darkblue')

plt.title('Number of dentist offices by population in ZIP areas')
plt.xlabel('Population in the ZIP area')
plt.ylabel('Number of Dental Offices')
for z in zip_data.index:
    plt.annotate(z, (zip_data['pop'][z], zip_data['dentists'][z]),
                textcoords='offset pixels', xytext=(-10,4),
                fontsize=7,
                alpha = zip_data['ratio'][z]/zip_data['ratio'].max()
                )
plt.show()

In [ ]:
# average ratio of dental offices per 10000 inabitanits

zip_data['dentists'].sum()/zip_data['pop'].sum() * 10000

In [ ]:
zip_data.plot(kind='scatter', x='income', y='dentists', figsize=(10, 6), color='darkblue')

plt.title('Number of dentist offices by median income in ZIP areas')
plt.xlabel('Median income in the ZIP area')
plt.ylabel('Number of Dental Offices')
for z in zip_data.index:
    plt.annotate(z, (zip_data['income'][z], zip_data['dentists'][z]),
                textcoords='offset pixels', xytext=(-10,4),
                fontsize=7,
                alpha = zip_data['ratio'][z]/zip_data['ratio'].max()
                )
plt.plot(x, regr.coef_[0][0]*x + regr.intercept_[0], '-r')
plt.show()

In [ ]:
import numpy as np
from sklearn import linear_model

# create multi linear regression model
# remove zip codes with NO dental offices for modelling
# zip_data.drop(zip_data[zip_data.dentists < 1].index, inplace=True)


regr = linear_model.LinearRegression()
x = np.asanyarray(zip_data[['pop','income']])
y = np.asanyarray(zip_data[['dentists']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)

In [ ]:
y_hat= regr.predict(zip_data[['pop', 'income']])
x = np.asanyarray(zip_data[['pop', 'income']])
y = np.asanyarray(zip_data[['dentists']])
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

In [ ]:
plt.plot(x, regr.coef_[0][0]*x + regr.intercept_[0], '-r')

In [ ]:
all_dentists = 0
for z in miami_dentists:
    all_dentists += len(miami_dentists[z].index)


In [ ]:
all_dentists

In [ ]:
# using predicted number of Offices (y_hat), 
# get number of ZIP codes where there are too few offices
result = np.rint(y-y_hat).astype(int)

In [ ]:
zip_data['result']=result

In [ ]:
zip_data.sort_values(by=['result','income', 'pop'])

In [ ]:
prospective = zip_data.sort_values(by=['result','income', 'pop'])[0:5].index

In [ ]:
prospective

In [ ]:
zip_map = r'Zip_Code.topojson' # topojson file
pop_df = pd.read_csv('population_by_zip.csv')

#yellow --> green color scale from 0 to maximum population
colorscale = branca.colormap.linear.YlGn_09.scale(0, pop_df.max()['pop'])
with open(zip_map, mode='r') as f:
    topo = json.load(f)
    
#find zip codes present in the topojson file
topo_zips = set()
for g in topo['objects']['Zip_Code']['geometries']:
    topo_zips.add(g['properties']['ZIP'])

#zip codes in population file
pop_zips=set(pop_df.index)

#add missing zip codes to population file (with value = 0)
#this is necessary for folium to parse all zip areas
for z in topo_zips - pop_zips:
    pop_df = pop_df.append({'zip':z, 'pop':0}, ignore_index=True)

pop_series = pop_df.set_index('zip')['pop']
def style_function(feature):
    pop = pop_series.get(int(feature['properties']['ZIP']), None)
    fillColor = '#black' if int(feature['properties']['ZIP']) in prospective else colorscale(pop)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': fillColor
    }

miami_area = [25.70, -80.208889]
m = folium.Map(
    location=miami_area,
    tiles='cartodbpositron',
    zoom_start=9
)
folium.TopoJson(
    topo,
    'objects.Zip_Code',
    style_function=style_function
).add_to(m)



In [ ]:
m

In [ ]:
for d in miami_dentists:
    try:
        add_dentists_to_map(miami_dentists[str(d)],m, add_label=', ZIP:'+str(d))
    except KeyError:
        continue

In [ ]:
m

In [ ]:
pop_df

In [ ]:
pop_zips=set(pop_df.index)

In [ ]:
pop_zips